In [1]:
!pip install fedex

In [2]:
import os
import sys

# Use the fedex directory included in the downloaded package instead of
# any globally installed versions.
#sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
from fedex.config import FedexConfig

# Change these values to match your testing account/meter number.
CONFIG_OBJ = FedexConfig(key='Z2mMBLkkq2fdRCtP',
                         password='zuXDi4RDuiDHnXZFoo9EsVdFl',
                         account_number='510087500',
                         meter_number='100420445',
                         freight_account_number='xxxxxxxxxxx',
                         use_test_server=True)

In [3]:
import os, sys

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [4]:
#!/usr/bin/env python
"""
This example shows how to use the FedEx RateRequest service.
The variables populated below represents the minimum required values.
You will need to fill all of these, or risk seeing a SchemaValidationError
exception thrown by suds.
TIP: Near the bottom of the module, see how to check the if the destination
     is Out of Delivery Area (ODA).
"""
import logging

from fedex.services.rate_service import FedexRateServiceRequest
from fedex.tools.conversion import sobject_to_dict

# Un-comment to see the response from Fedex printed in stdout.
#logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# This is the object that will be handling our request.
# We're using the FedexConfig object from example_config.py in this dir.
customer_transaction_id = "*** RateService Request v18 using Python ***"  # Optional transaction_id
rate_request = FedexRateServiceRequest(CONFIG_OBJ, customer_transaction_id=customer_transaction_id)

# If you wish to have transit data returned with your request you
# need to uncomment the following
# rate_request.ReturnTransitAndCommit = True

# This is very generalized, top-level information.
# REGULAR_PICKUP, REQUEST_COURIER, DROP_BOX, BUSINESS_SERVICE_CENTER or STATION
rate_request.RequestedShipment.DropoffType = 'REGULAR_PICKUP'

# See page 355 in WS_ShipService.pdf for a full list. Here are the common ones:
# STANDARD_OVERNIGHT, PRIORITY_OVERNIGHT, FEDEX_GROUND, FEDEX_EXPRESS_SAVER
# To receive rates for multiple ServiceTypes set to None.
#rate_request.RequestedShipment.ServiceType = 'None'

# What kind of package this will be shipped in.
# FEDEX_BOX, FEDEX_PAK, FEDEX_TUBE, YOUR_PACKAGING
rate_request.RequestedShipment.PackagingType = 'YOUR_PACKAGING'

# Shipper's address
rate_request.RequestedShipment.Shipper.Address.PostalCode = '29631'
rate_request.RequestedShipment.Shipper.Address.CountryCode = 'US'
rate_request.RequestedShipment.Shipper.Address.Residential = False

# Recipient address
rate_request.RequestedShipment.Recipient.Address.PostalCode = '27577'
rate_request.RequestedShipment.Recipient.Address.CountryCode = 'US'
# This is needed to ensure an accurate rate quote with the response.
# rate_request.RequestedShipment.Recipient.Address.Residential = True
# include estimated duties and taxes in rate quote, can be ALL or NONE
rate_request.RequestedShipment.EdtRequestType = 'NONE'

# Who pays for the rate_request?
# RECIPIENT, SENDER or THIRD_PARTY
rate_request.RequestedShipment.ShippingChargesPayment.PaymentType = 'SENDER'

package1_weight = rate_request.create_wsdl_object_of_type('Weight')
# Weight, in LB.
package1_weight.Value = 1.0
package1_weight.Units = "LB"

package1 = rate_request.create_wsdl_object_of_type('RequestedPackageLineItem')
package1.Weight = package1_weight
# can be other values this is probably the most common
package1.PhysicalPackaging = 'BOX'
# Required, but according to FedEx docs:
# "Used only with PACKAGE_GROUPS, as a count of packages within a
# group of identical packages". In practice you can use this to get rates
# for a shipment with multiple packages of an identical package size/weight
# on rate request without creating multiple RequestedPackageLineItem elements.
# You can OPTIONALLY specify a package group:
# package1.GroupNumber = 0  # default is 0
# The result will be found in RatedPackageDetail, with specified GroupNumber.
package1.GroupPackageCount = 1
# Un-comment this to see the other variables you may set on a package.
# print(package1)

# This adds the RequestedPackageLineItem WSDL object to the rate_request. It
# increments the package count and total weight of the rate_request for you.
rate_request.add_package(package1)

# If you'd like to see some documentation on the ship service WSDL, un-comment
# this line. (Spammy).
# print(rate_request.client)

# Un-comment this to see your complete, ready-to-send request as it stands
# before it is actually sent. This is useful for seeing what values you can
# change.
# print(rate_request.RequestedShipment)

# Fires off the request, sets the 'response' attribute on the object.

with HiddenPrints():
    rate_request.send_request()

# This will show the reply to your rate_request being sent. You can access the
# attributes through the response attribute on the request object. This is
# good to un-comment to see the variables returned by the FedEx reply.
# print(rate_request.response)

# This will convert the response to a python dict object. To
# make it easier to work with.
# from fedex.tools.conversion import basic_sobject_to_dict
# print(basic_sobject_to_dict(rate_request.response))

# This will dump the response data dict to json.
# from fedex.tools.conversion import sobject_to_json
# print(sobject_to_json(rate_request.response))

# Here is the overall end result of the query.
#print("HighestSeverity: {}".format(rate_request.response.HighestSeverity))
d = {}
# RateReplyDetails can contain rates for multiple ServiceTypes if ServiceType was set to None
for service in rate_request.response.RateReplyDetails:
    for rate_detail in service.RatedShipmentDetails:
        key = service.ServiceType
        d[key] = rate_detail.ShipmentRateDetail.TotalNetFedExCharge.Amount
# Check for warnings, this is also logged by the base class.



The origin state/province code has been changed.   (Error code: 819)
The destination state/province code has been changed.   (Error code: 820)


In [5]:
from collections import defaultdict
comp_rates = defaultdict(list)

for key in d.keys():
        comp_rates[key].append(d[key]- d['FEDEX_GROUND'])
        comp_rates[key].append((d[key]- d['FEDEX_GROUND'])*100/d[key])
        comp_rates[key].append(d[key])
   
    
    

In [8]:
comp_rates

defaultdict(list,
            {FIRST_OVERNIGHT: [70.05, 85.68807339449542, 81.75],
             PRIORITY_OVERNIGHT: [16.63, 58.70102364984116, 28.33],
             STANDARD_OVERNIGHT: [31.180000000000003,
              72.71455223880598,
              42.88],
             FEDEX_2_DAY_AM: [13.420000000000002, 53.4235668789809, 25.12],
             FEDEX_2_DAY: [11.780000000000001, 50.170357751277685, 23.48],
             FEDEX_EXPRESS_SAVER: [8.82, 42.98245614035088, 20.52],
             FEDEX_GROUND: [0.0, 0.0, 11.7]})

In [84]:
print("Using Ground shipment compared to other shipments will result in ")
print(" {:22s} {:6s} {:12s} {:12s} ".format("Service_Type","Cost","Cost_Saving","%Cost Saving"))
for values in comp_rates:
    print(" {:20s} {:6.2f} {:12.2f} {:12.2f} ".format(values,comp_rates[values][2],comp_rates[values][0],comp_rates[values][1]))

Using Ground shipment compared to other shipments will result in 
 Service_Type           Cost   Cost_Saving  %Cost Saving 
 FIRST_OVERNIGHT       81.75        70.05        85.69 
 PRIORITY_OVERNIGHT    28.33        16.63        58.70 
 STANDARD_OVERNIGHT    42.88        31.18        72.71 
 FEDEX_2_DAY_AM        25.12        13.42        53.42 
 FEDEX_2_DAY           23.48        11.78        50.17 
 FEDEX_EXPRESS_SAVER   20.52         8.82        42.98 
 FEDEX_GROUND          11.70         0.00         0.00 
